In [2]:
# https://wikidocs.net/60575

---
# 04. 소프트맥스 회귀 구현하기

In [3]:
# 소프트맥스를 low level과 F.cross_entropy를 사용해 구현해보자

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [5]:
# 훈련 데이터 선언
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)     # 선언할 때 부터 long 타입으로

# x에 4개의 특성이 있으며 총 8개의 샘플이 존재
# y에는 0,1,2의 레이블이 존재

In [6]:
# -----------------------------------------
# Low level로 구현하기

# 데이터 확인
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [7]:
# y데이터 원핫 인코딩 진행

y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [8]:
# x = (8,4)   y = (8,3) 이므로 W 행렬의 크기는 4*3 이어야 함

# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 0.1)

In [9]:
# 학습

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # hypothesis
    hypothesis = F.softmax(x_train.matmul(W) + b , dim=1)

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번 마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4}/{}  Cost {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000  Cost 1.098612
Epoch  100/1000  Cost 0.761050
Epoch  200/1000  Cost 0.689991
Epoch  300/1000  Cost 0.643229
Epoch  400/1000  Cost 0.604117
Epoch  500/1000  Cost 0.568255
Epoch  600/1000  Cost 0.533922
Epoch  700/1000  Cost 0.500291
Epoch  800/1000  Cost 0.466908
Epoch  900/1000  Cost 0.433507
Epoch 1000/1000  Cost 0.399962


In [10]:
# ----------------------------------------
# High level로 구현하기

In [11]:
# F.cross_entropy 는 그 자체로 소프트 맥스를 포함한다!

# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{}   Cost {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000   Cost 1.098612
Epoch  100/1000   Cost 0.761050
Epoch  200/1000   Cost 0.689991
Epoch  300/1000   Cost 0.643229
Epoch  400/1000   Cost 0.604117
Epoch  500/1000   Cost 0.568255
Epoch  600/1000   Cost 0.533922
Epoch  700/1000   Cost 0.500291
Epoch  800/1000   Cost 0.466908
Epoch  900/1000   Cost 0.433507
Epoch 1000/1000   Cost 0.399962


In [12]:
# ------------------------------
# 소프트맥스 회귀 nn.Module로 구현하기

In [13]:
# nn.linear()를 모델로 이용해 다중 분류를 만들어보자

model = nn.Linear(4, 3)
# 4개의 특성을 가지고 3개의 클래스로 분류
# input_dim = 4, output_dim 3

In [14]:
# cost는 F.cross_entropy()를 사용할 것임으로 softmax 함수를 따로 정의하지 않는다

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 20 == 0:
        print('Epoch {:4d}/{}  Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000  Cost: 1.616785
Epoch   20/1000  Cost: 0.904600
Epoch   40/1000  Cost: 0.778017
Epoch   60/1000  Cost: 0.722806
Epoch   80/1000  Cost: 0.686341
Epoch  100/1000  Cost: 0.658891
Epoch  120/1000  Cost: 0.636713
Epoch  140/1000  Cost: 0.617946
Epoch  160/1000  Cost: 0.601539
Epoch  180/1000  Cost: 0.586845
Epoch  200/1000  Cost: 0.573444
Epoch  220/1000  Cost: 0.561047
Epoch  240/1000  Cost: 0.549451
Epoch  260/1000  Cost: 0.538507
Epoch  280/1000  Cost: 0.528102
Epoch  300/1000  Cost: 0.518151
Epoch  320/1000  Cost: 0.508587
Epoch  340/1000  Cost: 0.499357
Epoch  360/1000  Cost: 0.490416
Epoch  380/1000  Cost: 0.481729
Epoch  400/1000  Cost: 0.473266
Epoch  420/1000  Cost: 0.465000
Epoch  440/1000  Cost: 0.456911
Epoch  460/1000  Cost: 0.448979
Epoch  480/1000  Cost: 0.441185
Epoch  500/1000  Cost: 0.433516
Epoch  520/1000  Cost: 0.425956
Epoch  540/1000  Cost: 0.418493
Epoch  560/1000  Cost: 0.411114
Epoch  580/1000  Cost: 0.403808
Epoch  600/1000  Cost: 0.396563
Epoch  6

In [16]:
# ------------------------------
# 4. 소프트맥스 회귀 클래스로 구현하기

In [17]:
# nn.Module을 상속받은 클래스로 구현하자

class SoftmaxClassfierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)  # input 4, output이 3

    def forward(self, x):
        return self.linear(x)

In [18]:
model = SoftmaxClassfierModel()

In [19]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번 마다 로그 출력
    if epoch % 20 == 0:
        print('Epoch {:4d}/{}  Cost {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000  Cost 2.637636
Epoch   20/1000  Cost 0.887444
Epoch   40/1000  Cost 0.773795
Epoch   60/1000  Cost 0.714015
Epoch   80/1000  Cost 0.675676
Epoch  100/1000  Cost 0.647903
Epoch  120/1000  Cost 0.625999
Epoch  140/1000  Cost 0.607701
Epoch  160/1000  Cost 0.591800
Epoch  180/1000  Cost 0.577594
Epoch  200/1000  Cost 0.564643
Epoch  220/1000  Cost 0.552655
Epoch  240/1000  Cost 0.541428
Epoch  260/1000  Cost 0.530818
Epoch  280/1000  Cost 0.520717
Epoch  300/1000  Cost 0.511043
Epoch  320/1000  Cost 0.501733
Epoch  340/1000  Cost 0.492736
Epoch  360/1000  Cost 0.484011
Epoch  380/1000  Cost 0.475525
Epoch  400/1000  Cost 0.467249
Epoch  420/1000  Cost 0.459159
Epoch  440/1000  Cost 0.451235
Epoch  460/1000  Cost 0.443458
Epoch  480/1000  Cost 0.435811
Epoch  500/1000  Cost 0.428281
Epoch  520/1000  Cost 0.420852
Epoch  540/1000  Cost 0.413514
Epoch  560/1000  Cost 0.406254
Epoch  580/1000  Cost 0.399061
Epoch  600/1000  Cost 0.391924
Epoch  620/1000  Cost 0.384834
Epoch  6